 Please install Python 3.10+ in your VM Workstation

This notebook needs to run on the local machine so please download the notebooks and execute

Login to your azure account az login - device

In [ ]:
# Install all required packages
import sys
import subprocess
import importlib

def install_package(package_name):
    try:
        __import__(package_name)
        print(f"✓ {package_name} is already installed")
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"✓ {package_name} installed successfully")

# Install essential packages
install_package('python-dotenv')
install_package('azure-identity')
install_package('azure-ai-ml')
install_package('tqdm')

# For safety, ensure azure-ai-ml is latest version
print("\nUpdating azure-ai-ml to latest version...")
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'azure-ai-ml', '--upgrade'])
print("✓ azure-ai-ml updated to latest version")

# Verify the Azure ML SDK installation
try:
    from azure.ai.ml import MLClient
    from azure.identity import DefaultAzureCredential
    
    # Check if az CLI is installed and user is logged in
    try:
        # This will fail if az CLI isn't installed or user isn't logged in
        credential = DefaultAzureCredential()
        print("✓ Azure authentication is available (DefaultAzureCredential)")
    except Exception as e:
        print(f"⚠️ Azure authentication issue: {str(e)}")
        print("Please run 'az login' in your terminal to authenticate")
        
except Exception as e:
    print(f"⚠️ Issue with Azure ML SDK: {str(e)}")
    print("Please ensure your Azure ML SDK installation is correct")

## Library Installation

First, let's install all the necessary libraries for model downloading and management. This includes:
- `python-dotenv` for environment variables management
- `azure-ai-ml` for Azure ML interactions
- `tqdm` for progress bars
- `azure-identity` for authentication

In [ ]:
! pip install azure-ai-ml -U  

## Azure Authentication

Before using this notebook, make sure you're authenticated with Azure by running:

```
az login --use-device-code
```

This will authenticate your session and allow the notebook to connect to your Azure resources.

In [ ]:
# Import required packages
import os
import time
import sys
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from tqdm.notebook import tqdm
from dotenv import load_dotenv

print("All packages successfully imported")

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from local.env file
print("Loading environment variables from local.env file...")
local_env_path = os.path.join(os.getcwd(), 'local.env')

if os.path.exists(local_env_path):
    load_dotenv(local_env_path)
    print(f"✓ Successfully loaded environment variables from {local_env_path}")
else:
    print(f"⚠️ Warning: local.env file not found at {local_env_path}")
    
    # Try to find sample.env and create local.env if needed
    sample_env_path = os.path.join(os.getcwd(), 'sample.env')
    if os.path.exists(sample_env_path):
        print(f"Found sample.env file. Creating a copy as local.env...")
        with open(sample_env_path, 'r') as sample_file:
            with open(local_env_path, 'w') as local_file:
                local_file.write(sample_file.read())
        print(f"✓ Created local.env file from sample. Please edit it with your actual credentials.")
        load_dotenv(local_env_path)
    else:
        print("❌ No environment files found. Creating a basic local.env file...")
        with open(local_env_path, 'w') as f:
            f.write("# Azure ML credentials\n")
            f.write("AZUREML_SUBSCRIPTION_ID=your-subscription-id\n")
            f.write("AZUREML_RESOURCE_GROUP=your-resource-group\n")
            f.write("AZUREML_WS_NAME=your-workspace-name\n")
        print("✓ Created a basic local.env file. Please edit it with your actual credentials.")

In [ ]:
# Get Azure ML credentials from environment variables
subscription_id = os.getenv('AZUREML_SUBSCRIPTION_ID')
resource_group = os.getenv('AZUREML_RESOURCE_GROUP')
workspace = os.getenv('AZUREML_WS_NAME')

# Check if any variable is missing
missing_vars = []
if not subscription_id:
    missing_vars.append("AZUREML_SUBSCRIPTION_ID")
if not resource_group:
    missing_vars.append("AZUREML_RESOURCE_GROUP")
if not workspace:
    missing_vars.append("AZUREML_WS_NAME")

if missing_vars:
    print(f"❌ Error: The following required environment variables are missing: {', '.join(missing_vars)}")
    print("Please edit your local.env file to include these variables with valid values.")
else:
    # Print values for debugging
    print(f"✓ Subscription ID: {subscription_id}")
    print(f"✓ Resource Group: {resource_group}")
    print(f"✓ Workspace: {workspace}")
    
    try:
        # Create ML client with the credentials
        print("Connecting to Azure ML...")
        ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
        print("✓ Successfully connected to Azure ML workspace")
    except Exception as e:
        print(f"❌ Error connecting to Azure ML: {str(e)}")
        print("Please check your credentials and make sure you're logged in (az login)")

In [ ]:
# List available models in the workspace
try:
    print("Listing available models in workspace...")
    models = list(ml_client.models.list())
    
    if models:
        print(f"Found {len(models)} models in the workspace:")
        for model in models:
            print(f" - {model.name} (version: {model.version})")
    else:
        print("No models found in the workspace.")
        print("Please verify that you have the correct workspace and permissions.")
        
except Exception as e:
    print(f"❌ Error listing models: {str(e)}")
    print("This could be due to incorrect credentials or Azure ML workspace information.")
    print("Please verify your local.env file has the correct values.")

## Download the Fine-Tuned Model

Now we'll download the fine-tuned Phi-4-Mini model from Azure ML Model Registry. This process will:

1. Connect to Azure ML using our credentials
2. Download the ONNX model (int4 quantized for CPU)
3. Display a progress bar during download
4. Show the download location and model size when complete

After downloading, you'll be able to use this model locally for inference.

In [ ]:
# Define model name and version
model_name = "fine-tuning-phi-4-mini-onnx-int4-cpu"
model_version = 1

try:
    print(f"\nStarting download of model: {model_name} (version {model_version})\n")
    
    # Create a progress bar
    with tqdm(total=100, desc="Downloading model", unit="%") as pbar:
        try:
            # Start a background thread to download the model
            # Note: Some Azure ML SDK versions use download_async, others use download with the same behavior
            try:
                download_future = ml_client.models.download_async(name=model_name, version=model_version)
            except AttributeError:
                # Fall back to regular download method if download_async is not available
                print("Using synchronous download method (older SDK version detected)")
                download_path = ml_client.models.download(name=model_name, version=model_version)
                pbar.n = 100
                pbar.refresh()
                print(f"\nModel downloaded successfully to: {download_path}")
                # Skip the rest of the async handling
                raise StopIteration()
            
            # Monitor the download progress
            while not download_future.done():
                time.sleep(0.5)
                # Update progress bar (approximate since we don't have actual progress info from the API)
                pbar.update(1)
                pbar.refresh()
                if pbar.n >= 95:  # Cap at 95% until confirmed complete
                    pbar.n = 95
                    pbar.refresh()
    
            # Get the result
            download_path = download_future.result()
            pbar.n = 100  # Set to 100% when complete
            pbar.refresh()
            
            print(f"\nModel downloaded successfully to: {download_path}")
            
        except StopIteration:
            # This is used to skip the rest of the async handling
            pass
        except Exception as e:
            print(f"\n❌ Error downloading model: {str(e)}")
            raise
    
    # Calculate and display the size of the downloaded model
    if 'download_path' in locals() and os.path.exists(download_path):
        total_size = 0
        for path, dirs, files in os.walk(download_path):
            for f in files:
                fp = os.path.join(path, f)
                total_size += os.path.getsize(fp)
    
        # Convert to appropriate size format (bytes, KB, MB, GB)
        def format_size(size_bytes):
            if size_bytes < 1024:
                return f"{size_bytes} bytes"
            elif size_bytes < 1024**2:
                return f"{size_bytes/1024:.2f} KB"
            elif size_bytes < 1024**3:
                return f"{size_bytes/(1024**2):.2f} MB"
            else:
                return f"{size_bytes/(1024**3):.2f} GB"
    
        print(f"Total model size: {format_size(total_size)}")
    else:
        print("❌ Model download failed or path not found")
        
except Exception as e:
    print(f"\n❌ Error in model download process: {str(e)}")
    print("Please check your Azure ML permissions and make sure the model exists in your workspace.")
    print("You can list available models with ml_client.models.list() to verify the model name and version.")